<a href="https://colab.research.google.com/github/BryanMui1/HandTracker/blob/videotrackdemo/videoTracking1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Using cached PyYAML-5.1.tar.gz (274 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-uxc13put
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-uxc13put
  Resolved https://github.com/facebookresearch/detectron2.git to commit 4e80df1e58901557e2824ce3b488d30209a9be33
  Preparing metadata (setup.py) ... done


In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


In [ ]:
torch.cuda.is_available()

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
path="/content/drive/MyDrive/videoTrackingFiles/"
os.chdir(path)
os.listdir(path)

['output',
 'faster_rcnn_X_101_32x8d_FPN_3x_100DOH.yaml',
 'models',
 'viz',
 'Base-RCNN-FPN.yaml',
 'origamitrim.mp4',
 'output1.avi']

In [ ]:
#construts the frames, don't run more than once!!
import cv2

capture = cv2.VideoCapture('origamitrim.mp4')
frameNr = 0

while(True):
  success, frame = capture.read()
  if success:
    cv2.imwrite(f'output/frame_{frameNr}.jpg', frame, [cv2.IMWRITE_JPEG_QUALITY, 100])
  else:
    break
  frameNr += 1

capture.release()

In [ ]:
len(os.listdir('/content/drive/MyDrive/videoTrackingFiles/output'))

606

In [ ]:
torch.cuda.empty_cache()

In [ ]:
print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

torch.cuda.memory_allocated: 0.000000GB
torch.cuda.memory_reserved: 0.416016GB
torch.cuda.max_memory_reserved: 13.683594GB


In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved

import logging
from collections import OrderedDict
from tqdm import tqdm
import torch
import cv2
import random

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer

from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch
from detectron2.evaluation import (
    DatasetEvaluators,
    PascalVOCDetectionEvaluator,
    verify_results,
)
from detectron2.modeling import GeneralizedRCNNWithTTA
from detectron2.data.datasets import register_coco_instances
from detectron2.data.datasets.pascal_voc import register_pascal_voc


def main():
    # load cfg and model
    cfg = get_cfg()
    cfg.merge_from_file("faster_rcnn_X_101_32x8d_FPN_3x_100DOH.yaml")
    cfg.MODEL.WEIGHTS = 'models/model_0529999.pth' # add model weight here
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # 0.5 , set the testing threshold for this model
    cfg.MODEL.DEVICE = "cuda"

    # predict
    predictor = DefaultPredictor(cfg)

    # output
    frameNr = 0
    maxFrame = len(os.listdir('/content/drive/MyDrive/videoTrackingFiles/output'))

    for _ in range(0, maxFrame):
      im = cv2.imread(f'output/frame_{frameNr}.jpg')
      outputs = predictor(im)
      v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
      v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      cv2.imwrite(f'./viz/outputs/frame_{frameNr}_out.jpg', v.get_image()[:, :, ::-1])
      frameNr += 1

      # print(outputs["instances"].pred_classes)
      # print(outputs["instances"].pred_boxes)


if __name__ == '__main__':
    main()

In [ ]:
print(len(os.listdir('./viz/outputs')))
cv2.destroyAllWindows()

606


In [ ]:
import numpy as np
import cv2

images = [img for img in os.listdir('./viz/outputs/') if img.endswith(".jpg")]
print(len(images))

frame = cv2.imread(os.path.join('./viz/outputs/', images[0]))
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter("model_out.avi", fourcc, 30, (width, height))


for image in images:
  video.write(cv2.imread(os.path.join('./viz/outputs/', image)))

cv2.destroyAllWindows()
video.release()

606
